### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

In [76]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [77]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [78]:
# Use this cell to find the following information:
# number of movies
print(f"Number of movies: {len(movies['movie_id'].unique())}")
# number of ratings
print(f"Number of ratings: {reviews[~reviews['rating'].isna()].shape[0]}")
# number of different genres
genres = set()
for r in movies['genre'].dropna().str.split('|'):
    for g in r:
        genres.add(g)
print(f"Number of different genres: {len(genres)}")
# number of unique users
print(f"Number of unique users: {len(reviews['user_id'].unique())}")
# number of missing ratings
print(f"Number of missing ratings: {reviews['rating'].isna().sum()}")
# the `average`, `min`, and `max` ratings given
print(f"Ratings: min = {reviews['rating'].min()}, max = {reviews['rating'].max()}, avg = {reviews['rating'].mean()}")

Number of movies: 35479
Number of ratings: 100001
Number of different genres: 28
Number of unique users: 8022
Number of missing ratings: 0
Ratings: min = 0, max = 10, avg = 7.397666023339767


In [79]:
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [80]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 8022
b = 10
c = 7
d = 35479
e = 15
f = 0
g = 4
h = 100001
i = 28

# Guide: replace "ENTER YOUR ANSWER HERE" with the matching letter above.
# For example, 'The number of movies in the dataset': d,

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [81]:
# Pull the date from the title and create new column
movies['year'] = movies['movie'].str.extract(r'\((\d{4})\)')

In [82]:
# Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
movies["1800's"] = movies['year'].apply(lambda year_text: 1 if int(year_text) < 1900 else 0)
movies["1900's"] = movies['year'].apply(lambda year_text: 1 if int(year_text) > 1899 and int(year_text) < 2000 else 0)
movies["2000's"] = movies['year'].apply(lambda year_text: 1 if int(year_text) > 1900 else 0)

In [83]:
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [84]:
# Dummy column the genre with 1's and 0's for each genre
genre_dict = {g:i for i,g in enumerate(genres)}

def get_genre_vector(genre_list):
    genre_vector = [0]*len(genres)
    try:
        for g in genre_list:
            genre_vector[genre_dict[g]] = 1
    except:
        pass
    
    return genre_vector

genre_dummies_series = movies.genre.str.split('|').apply(get_genre_vector)

In [85]:
genre_dummies_df = pd.DataFrame(genre_dummies.apply(pd.Series))
genre_dummies_df.columns = list(genre_dict.keys())

In [86]:
movies = pd.concat([movies, genre_dummies_df], axis=1)

In [89]:
movies.head()

,movie_id,movie,genre,year,1800's,1900's,2000's,Mystery,Thriller,Comedy,...,Family,Game-Show,Biography,Short,Crime,Fantasy,Reality-TV,News,Romance,Action
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [93]:
# Create a date out of time stamp
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')
reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [94]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 21:00:50
1           1        2    208092       5  1586466072  2020-04-09 21:01:12
2           2        2    358273       9  1579057827  2020-01-15 03:10:27
3           3        2  10039344       5  1578603053  2020-01-09 20:50:53
4           4        2   6751668       9  1578955697  2020-01-13 22:48:17
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Documentary  Adventure  \
0  Docume